In [ ]:
conda create -n qwen3 python=3.10 -y
conda activate qwen3

pip install torch==2.6.0 torchvision==0.21.0 torchaudio==2.6.0 --index-url https://download.pytorch.org/whl/cu126

pip install "transformers>=4.51.0" accelerate tokenizers safetensors sentencepiece einops protobuf numpy scipy

pip install "vllm==0.9.0"


In [4]:
# 验证安装
import torch, transformers
print("Torch:", torch.__version__, "CUDA available:", torch.cuda.is_available())
print("Transformers:", transformers.__version__)
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))



Torch: 2.7.0+cu126 CUDA available: True
Transformers: 4.55.4
GPU: NVIDIA vGPU-48GB


In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/root/autodl-tmp/Qwen/Qwen3-8B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# prepare the model input
prompt = "Give me a short introduction to large language model."
messages = [
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=16384
)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]):].tolist() 

content = tokenizer.decode(output_ids, skip_special_tokens=True)

print("content:", content)

Loading checkpoint shards: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]


content: <think>
Okay, the user asked for a short introduction to large language models. Let me start by recalling what I know about them. Large language models are a type of AI that can understand and generate human-like text. They're trained on vast amounts of data, right? I should mention things like natural language processing and machine learning.

Wait, maybe I should define what a large language model is first. They're part of the broader field of NLP, but they're specifically large-scale. The key points are their size, the training data, and their capabilities. Oh, and they can do tasks like answering questions, writing stories, coding, etc. 

I need to make sure I explain why they're called "large." Probably because they have a huge number of parameters, which allows them to capture complex patterns in language. Also, they're trained on diverse data sources, which helps them handle various tasks. 

But I should keep it concise since the user wants a short intro. Maybe start wi

In [ ]:
# vllm服务调用

vllm serve /root/autodl-tmp/Qwen/Qwen3-8B --dtype auto --port 6006  --max_model_len 8784 --gpu_memory_utilization 0.6

# 端口映射 本地上打开8000端口 注意复制密码
ssh -CNg -L 8000:127.0.0.1:6006 root@connect.bjb1.seetacloud.com -p 19434

nviHq5jZ1kQW


In [ ]:
import requests
import json
from PIL import Image
import base64
import time
 
def encode_image(image_path):       # 编码本地图片的函数
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
 
start = time.time()
# 1.url
url = 'http://localhost:8000/v1/chat/completions'
 
 
# 2.data
## 2.1如果server.py启动，用这个data
data = {"model": "/root/autodl-tmp/Qwen/Qwen3-8B",
        "messages": [{"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},     # 系统命令，一般不要改
                     {"role": "user",
                      "content": "Tell me something about large language models."}],    # 用户命令，一般改这里
        "temperature": 0.7,"top_p": 0.8,"repetition_penalty": 1.05,"max_tokens": 1024}
 
 
# 3.将字典转换为 JSON 字符串
json_payload = json.dumps(data)
 
 
# 4.发送 POST 请求
headers = {'Content-Type': 'application/json'}
response = requests.post(url, data=json_payload, headers=headers)
 

# 5.打印响应内容
print(response)
print(response.json().get("choices", [])[0].get("message", []).get("content", []))
# print(response.json())        # 调试用
 
print("\n总时间：", time.time()-start, "秒")
 

ConnectionError: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /v1/chat/completions (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb38e456aa0>: Failed to establish a new connection: [Errno 111] Connection refused'))